# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 6882, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 6882 (delta 18), reused 0 (delta 0), pack-reused 6822 (from 3)
Receiving objects: 100% (6882/6882), 412.47 MiB | 21.06 MiB/s, done.
Resolving deltas: 100% (3749/3749), done.
Updating files: 100% (506/506), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.7 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprec

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithKFCV/SLIM'

np.random.seed(42)

## Import the repository **RECsys_Challenge2024**

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'SLIM_BPR',
    'n_folds': 5,
    'metric': 'MAP',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SLIM_BPR_MAP.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/working/RECsys_Challenge2024/Dataset/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/working/RECsys_Challenge2024/Dataset/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_k_folds import split_train_k_folds

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

folds = split_train_k_folds(URM_all, k=config['n_folds'])

In [12]:
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python

def objective_function_SLIM_BPR(optuna_trial):

    full_hyperp = {
                    "topK": optuna_trial.suggest_int("topK", 0, 500),
                    "epochs": optuna_trial.suggest_int("epochs", 1, 50),
                    "lambda_i": optuna_trial.suggest_float("lambda_i", 1e-5, 1e-1, log=True),
                    "lambda_j": optuna_trial.suggest_float("lambda_j", 1e-5, 1e-1, log=True),
                    "learning_rate": optuna_trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
                }
    
    validation_results = []
    
    for fold_idx, (URM_train_fold, URM_validation_fold) in enumerate(folds):
    
        recommender_instance = SLIM_BPR_Python(URM_train_fold)
        recommender_instance.fit(**full_hyperp)
        
        evaluator = EvaluatorHoldout(URM_validation_fold, cutoff_list=[10])
        result_df, _ = evaluator.evaluateRecommender(recommender_instance)
        
        validation_results.append(result_df.loc[10]["MAP"])
    
    return np.mean(validation_results)

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_SLIM_BPR, n_trials=20, timeout=11*3600)

[I 2025-01-06 13:23:59,468] Using an existing study with name 'hyperparameters_tuning_SLIM_BPR_MAP' instead of creating a new one.


Epoch 1, Iteration 35736 in 11.04 seconds. Samples per second 3238.35
Epoch 2, Iteration 35736 in 7.95 seconds. Samples per second 4496.03
Epoch 3, Iteration 35736 in 6.37 seconds. Samples per second 5614.08
Epoch 4, Iteration 35736 in 5.40 seconds. Samples per second 6621.45
Epoch 5, Iteration 35736 in 4.65 seconds. Samples per second 7688.05
Epoch 6, Iteration 35736 in 4.42 seconds. Samples per second 8089.36
Epoch 7, Iteration 35736 in 4.01 seconds. Samples per second 8911.59
Epoch 8, Iteration 35736 in 3.74 seconds. Samples per second 9553.23
Epoch 9, Iteration 35736 in 3.62 seconds. Samples per second 9870.90
Epoch 10, Iteration 35736 in 3.47 seconds. Samples per second 10304.69
Epoch 11, Iteration 35736 in 3.40 seconds. Samples per second 10500.32
Epoch 12, Iteration 35736 in 3.43 seconds. Samples per second 10415.60
Epoch 13, Iteration 35736 in 3.31 seconds. Samples per second 10796.55
Epoch 14, Iteration 35736 in 3.26 seconds. Samples per second 10962.05
Epoch 15, Iteration 357

[I 2025-01-06 13:51:40,034] Trial 41 finished with value: 0.040179893231660754 and parameters: {'topK': 22, 'epochs': 39, 'lambda_i': 0.012002995832800612, 'lambda_j': 3.897810787222435e-05, 'learning_rate': 0.09581897940476075}. Best is trial 25 with value: 0.04090163381949502.


Epoch 1, Iteration 35736 in 10.40 seconds. Samples per second 3437.30
Epoch 2, Iteration 35736 in 7.60 seconds. Samples per second 4700.95
Epoch 3, Iteration 35736 in 5.96 seconds. Samples per second 5999.93
Epoch 4, Iteration 35736 in 5.08 seconds. Samples per second 7029.93
Epoch 5, Iteration 35736 in 4.52 seconds. Samples per second 7901.32
Epoch 6, Iteration 35736 in 4.19 seconds. Samples per second 8522.55
Epoch 7, Iteration 35736 in 3.94 seconds. Samples per second 9072.03
Epoch 8, Iteration 35736 in 3.84 seconds. Samples per second 9310.50
Epoch 9, Iteration 35736 in 3.83 seconds. Samples per second 9329.64
Epoch 10, Iteration 35736 in 3.53 seconds. Samples per second 10132.79
Epoch 11, Iteration 35736 in 3.44 seconds. Samples per second 10386.62
Epoch 12, Iteration 35736 in 3.40 seconds. Samples per second 10508.71
Epoch 13, Iteration 35736 in 3.32 seconds. Samples per second 10761.42
Epoch 14, Iteration 35736 in 3.31 seconds. Samples per second 10800.47
Epoch 15, Iteration 357

[I 2025-01-06 14:18:13,245] Trial 42 finished with value: 0.0002664826418719655 and parameters: {'topK': 0, 'epochs': 38, 'lambda_i': 0.010760526931868639, 'lambda_j': 2.647533217883142e-05, 'learning_rate': 0.04365119617520986}. Best is trial 25 with value: 0.04090163381949502.


...

[I 2025-01-06 21:59:05,114] Trial 58 finished with value: 0.035223853701749505 and parameters: {'topK': 144, 'epochs': 49, 'lambda_i': 0.0006479079657965158, 'lambda_j': 0.00028865427857367265, 'learning_rate': 0.07482277759234411}. Best is trial 54 with value: 0.04209128232743534.


Epoch 1, Iteration 35736 in 10.11 seconds. Samples per second 3534.63
Epoch 2, Iteration 35736 in 7.45 seconds. Samples per second 4794.29
Epoch 3, Iteration 35736 in 6.12 seconds. Samples per second 5839.70
Epoch 4, Iteration 35736 in 5.11 seconds. Samples per second 6996.72
Epoch 5, Iteration 35736 in 4.58 seconds. Samples per second 7802.76
Epoch 6, Iteration 35736 in 4.22 seconds. Samples per second 8473.12
Epoch 7, Iteration 35736 in 3.91 seconds. Samples per second 9129.04
Epoch 8, Iteration 35736 in 3.76 seconds. Samples per second 9500.03
Epoch 9, Iteration 35736 in 3.57 seconds. Samples per second 10010.08
Epoch 10, Iteration 35736 in 3.53 seconds. Samples per second 10126.85
Epoch 11, Iteration 35736 in 3.58 seconds. Samples per second 9990.40
Epoch 12, Iteration 35736 in 3.41 seconds. Samples per second 10472.47
Epoch 13, Iteration 35736 in 3.34 seconds. Samples per second 10697.37
Epoch 14, Iteration 35736 in 3.37 seconds. Samples per second 10610.37
Epoch 15, Iteration 357

[I 2025-01-06 22:22:28,997] Trial 59 finished with value: 0.03070893034447749 and parameters: {'topK': 234, 'epochs': 19, 'lambda_i': 0.003763089430980978, 'lambda_j': 0.0070001681381142485, 'learning_rate': 0.038619909781196574}. Best is trial 54 with value: 0.04209128232743534.


Epoch 1, Iteration 35736 in 10.24 seconds. Samples per second 3489.73
Epoch 2, Iteration 35736 in 7.41 seconds. Samples per second 4820.17
Epoch 3, Iteration 35736 in 5.96 seconds. Samples per second 5999.54
Epoch 4, Iteration 35736 in 5.04 seconds. Samples per second 7087.87
Epoch 5, Iteration 35736 in 4.62 seconds. Samples per second 7733.51
Epoch 6, Iteration 35736 in 4.31 seconds. Samples per second 8292.45
Epoch 7, Iteration 35736 in 4.09 seconds. Samples per second 8731.50
Epoch 8, Iteration 35736 in 3.78 seconds. Samples per second 9447.33
Epoch 9, Iteration 35736 in 3.61 seconds. Samples per second 9899.85
Epoch 10, Iteration 35736 in 3.54 seconds. Samples per second 10101.34
Epoch 11, Iteration 35736 in 3.39 seconds. Samples per second 10526.77
Epoch 12, Iteration 35736 in 3.34 seconds. Samples per second 10683.61
Epoch 13, Iteration 35736 in 3.40 seconds. Samples per second 10500.84
Epoch 14, Iteration 35736 in 3.23 seconds. Samples per second 11068.82
Epoch 15, Iteration 357

[I 2025-01-06 22:47:21,213] Trial 60 finished with value: 0.03306059768684737 and parameters: {'topK': 47, 'epochs': 36, 'lambda_i': 0.024827949166330205, 'lambda_j': 0.0014621160249445268, 'learning_rate': 0.006089735001563951}. Best is trial 54 with value: 0.04209128232743534.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = SLIM_BPR_Python(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

Epoch 1, Iteration 35736 in 10.99 seconds. Samples per second 3251.12
Epoch 2, Iteration 35736 in 7.80 seconds. Samples per second 4581.44
Epoch 3, Iteration 35736 in 6.29 seconds. Samples per second 5684.35
Epoch 4, Iteration 35736 in 5.30 seconds. Samples per second 6736.60
Epoch 5, Iteration 35736 in 4.71 seconds. Samples per second 7595.25
Epoch 6, Iteration 35736 in 4.40 seconds. Samples per second 8117.23
Epoch 7, Iteration 35736 in 4.11 seconds. Samples per second 8688.60
Epoch 8, Iteration 35736 in 3.82 seconds. Samples per second 9363.42
Epoch 9, Iteration 35736 in 3.79 seconds. Samples per second 9421.27
Epoch 10, Iteration 35736 in 3.60 seconds. Samples per second 9927.71
Epoch 11, Iteration 35736 in 3.49 seconds. Samples per second 10237.01
Epoch 12, Iteration 35736 in 3.47 seconds. Samples per second 10310.13
Epoch 13, Iteration 35736 in 3.40 seconds. Samples per second 10505.01
Epoch 14, Iteration 35736 in 3.40 seconds. Samples per second 10515.15
Epoch 15, Iteration 3573

# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/working/RECsys_Challenge2024/Dataset/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_SLIM_BPR_MAP.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/best_params_SLIM_BPR_MAP.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/history_SLIM_BPR_MAP.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/Submission/submission_SLIM_BPR_MAP.csv' updated successfully.
